In [1]:
from ElipticCurve import EllipticCurve
import time
import tracemalloc

In [2]:
def measure_time(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    execution_time = end_time - start_time
    return result, execution_time

Тестування часу і пам'яті, необхідну для створення ключів

In [5]:
with open("zapovit.txt", 'r', encoding="utf-8") as file:
    small_text = file.read()
with open("kateryna.txt", 'r', encoding="utf-8") as file:
    medium_text = file.read()
with open("tygrolovy.txt", 'r', encoding="utf-8") as file:
    big_text = file.read()

public_key = None
private_key = None
dsa = None
time_used = None
memory_usage = None

ec = EllipticCurve()
tracemalloc.start()
(private_key, public_key), time_used = measure_time(ec.generate_keys)
memory_usage = tracemalloc.get_traced_memory()[1] / (1024 ** 2)
tracemalloc.clear_traces()
print(public_key, private_key)
print(f'The time used to generate keys: {time_used}s')
print(f'The memory used to create keys: {memory_usage} MB')

(8928723677400577649925073127216718283001234342290929163090448522806218250254, 24063634504411369444004388328406893104921309130290362409525029265453700107952) 98746280228231762573942584406806761399288439163733007552775784393906454535145
The time used to generate keys: 0.8300788402557373s
The memory used to create keys: 3.3558034896850586 MB


Список зі зміненими текстами, щоб перевірити алгоритм.

In [7]:
changed = [small_text + '&', medium_text + 'і', big_text + '  ']

Тестування ефективності алгоритму на повідомленнях різної довжини.

In [9]:
for i, text1 in enumerate([(small_text, 'small'), (medium_text, 'medium'), (big_text, 'big')]):
    text, name = text1
    tracemalloc.start()
    signature, encoding_time = measure_time(ec.sign, text, public_key, private_key)
    encoding_memory_usage = tracemalloc.get_traced_memory()[1] / (1024 ** 2)  # Переведення в мегабайти
    tracemalloc.clear_traces()
    tracemalloc.start()
    right_answer, verification_time = measure_time(ec.verify, text, signature[0], signature[1], public_key)
    decoding_memory_usage = tracemalloc.get_traced_memory()[1] / (1024 ** 2)  # Переведення в мегабайти
    tracemalloc.clear_traces()
    tracemalloc.start()
    answer, verification_time2 = measure_time(ec.verify, changed[i], signature[0], signature[1], public_key)
    decoding_memory_usage2 = tracemalloc.get_traced_memory()[1] / (1024 ** 2)  # Переведення в мегабайти
    tracemalloc.clear_traces()
    print(f"Correctness: \nAnswer for right text is {right_answer}\nAnswer for changed text is {answer}")
    print(f"The time used for creating signature for {name} text is {encoding_time}, memory - {encoding_memory_usage} MB")
    print(f"The time used for verification of {name} text is {(verification_time+verification_time2)/2}, memory - {(decoding_memory_usage+decoding_memory_usage2)/2} MB")

Correctness: 
Answer for right text is True
Answer for changed text is False
The time used for creating signature for small text is 0.8649799823760986, memory - 0.16614818572998047 MB
The time used for verification of small text is 2.580408215522766, memory - 0.003562450408935547 MB
Correctness: 
Answer for right text is True
Answer for changed text is False
The time used for creating signature for medium text is 0.8998403549194336, memory - 0.1156320571899414 MB
The time used for verification of medium text is 2.6326366662979126, memory - 0.11504554748535156 MB
Correctness: 
Answer for right text is True
Answer for changed text is False
The time used for creating signature for big text is 0.8494484424591064, memory - 3.303110122680664 MB
The time used for verification of big text is 2.5748157501220703, memory - 3.302522659301758 MB


In [10]:
from encode_photo import encode_image_to_base64
from decode_photo import decode_base64_file_to_image

small_photo_encoded = encode_image_to_base64('very_small_image.jpeg')
large_photo_encoded = encode_image_to_base64('photo.jpg')

In [11]:
changed = [small_photo_encoded + '8', large_photo_encoded + 'w']

Тестування ефективності повідомлення у верифікації фото

In [13]:
for i, text1 in enumerate([(small_photo_encoded, 'small'), (large_photo_encoded, 'medium')]):
    text, name = text1
    tracemalloc.start()
    signature, encoding_time = measure_time(ec.sign, text, public_key, private_key)
    encoding_memory_usage = tracemalloc.get_traced_memory()[1] / (1024 ** 2)  # Переведення в мегабайти
    tracemalloc.clear_traces()
    tracemalloc.start()
    right_answer, verification_time = measure_time(ec.verify, text, signature[0], signature[1], public_key)
    decoding_memory_usage = tracemalloc.get_traced_memory()[1] / (1024 ** 2)  # Переведення в мегабайти
    tracemalloc.clear_traces()
    tracemalloc.start()
    answer, verification_time2 = measure_time(ec.verify, changed[i], signature[0], signature[1], public_key)
    decoding_memory_usage2 = tracemalloc.get_traced_memory()[1] / (1024 ** 2)  # Переведення в мегабайти
    tracemalloc.clear_traces()
    print(f"Correctness: \nAnswer for right text is {right_answer}\nAnswer for changed text is {answer}")
    print(f"The time used for creating signature for {name} text is {encoding_time}, memory - {encoding_memory_usage} MB")
    print(f"The time used for verification of {name} text is {(verification_time+verification_time2)/2}, memory - {(decoding_memory_usage+decoding_memory_usage2)/2} MB")

Correctness: 
Answer for right text is True
Answer for changed text is False
The time used for creating signature for small text is 0.9005239009857178, memory - 0.2805509567260742 MB
The time used for verification of small text is 2.6983996629714966, memory - 0.19435834884643555 MB
Correctness: 
Answer for right text is True
Answer for changed text is False
The time used for creating signature for medium text is 0.8911404609680176, memory - 0.5860795974731445 MB
The time used for verification of medium text is 2.6840375661849976, memory - 0.5854907035827637 MB
